# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [ ]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144
#from static_grader import grader

import pandas as pd
import numpy as np
import gzip

# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)

# load the 2016 data
with gzip.open('./dw-data/201606scripts_sample.csv.gz', 'rb') as f:
    scripts2016 = pd.read_csv(f)

col_names=['code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f)
    
practices.columns = col_names

with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [ ]:
scripts.describe()

In [ ]:
scripts.describe().act_cost

In [ ]:
def stats_df(df):
    df_stats = df.describe() 
    list_df_stats = []
    for cols in df_stats: 
        total_df = df[str(cols)].sum()
        mean_df = df_stats[str(cols)][1]
        std_df = df_stats[str(cols)][2]
        q25_df = df_stats[str(cols)][4]
        median_df = df_stats[str(cols)][5]
        q75_df = df_stats[str(cols)][6]
        list_df_stats.append((str(cols),
                              (total_df, mean_df, std_df, q25_df, median_df, q75_df )
                             )
                            )
    return list_df_stats

In [ ]:
list_test_stats = stats_df(scripts)

In [ ]:
list_test_stats

In [ ]:
summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [ ]:
grader.score.dw__summary_statistics(list_test_stats)

## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [ ]:
bnf_names_grouped = scripts.groupby(by = 'bnf_name', sort=True).sum()

In [ ]:
bnf_names_grouped['items'][bnf_names_grouped['items'] == 218583]

In [ ]:
bnf_names_grouped['items']

In [ ]:
most_common_item = [("Omeprazole_Cap E/C 20mg", 218583)]

In [ ]:
#grader.score.dw__most_common_item(most_common_item)

## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [ ]:
practices.head()

In [ ]:
scripts.head()

In [ ]:
regions = scripts

'''
There can be multiple post codes for a practice. 
the first step is to take the alphabetically first 
"post_code" for each "code" in Practice dataset and 
then go on with your current logic.
'''
## What groupby.first is doing? ( taking first alphabetical post code for code? )
practiceFrame = practices.sort_values('post_code').groupby('code').first().reset_index()

regions = regions.merge(practiceFrame, left_on='practice', right_on='code')
region_groups = regions.groupby('post_code') 

items = []
post_codes = sorted(regions['post_code'].unique())
itert = 0

In [ ]:
for code in post_codes:
    data = region_groups.get_group(code) ## getting group by post_code
    group_list = data['bnf_name'].unique() ## checking for unique names 
    group_cont = data.groupby('bnf_name') ## groupping by names

    total_items = sum(data['items']) ## total items 
    bnfname = ""
    group_items = 0

    for group in group_list:
        group_data = group_cont.get_group(group)

        total = sum(group_data['items'])
        
        if group_items < total:
            bnfname = group
            group_items = total
        elif group_items == total:
            if bnfname > group:
                bnfname = group

    items.append((code, bnfname, float("{:.10f}".format(group_items/total_items))))
    itert += 1
    if itert >= 100:
        break
        

In [ ]:
grader.score.dw__items_by_region(items)

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [ ]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144
from static_grader import grader

import pandas as pd
import numpy as np
import gzip

# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)

# load the 2016 data
with gzip.open('./dw-data/201606scripts_sample.csv.gz', 'rb') as f:
    scripts2016 = pd.read_csv(f)

col_names=['code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f)
    
practices.columns = col_names

with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)


In [ ]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [ ]:
scripts.head()

In [ ]:
chem.head()

In [ ]:
practices.head()

In [ ]:
# What you may opt to do:
# 1: Use opiods list opiods = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine'] to flag rows with opiod prescription 'True' or 'False'
# 2: Create a pattern: pattern = '|'.join(opiods)
# Output: 'morphine|oxycodone|methadone|fentanyl|pethidine|buprenorphine|propoxyphene|codeine'
# 3: Now use this pattern to flag rows in chem dataframe flag = chem['NAME'].str.contains(pattern, case=False)
    
# 4: Filter out codes from the chem dataframe using flag opiod_codes = chem[flag]['CHEM SUB']
    
# 5: Create 'opiods' column in scripts and fill it with 1 if scripts['bnf_code]' is in opiod_codes or 0 if scripts['bnf_code]' is not in opiod_codes
# scripts['opiods'] = scripts['bnf_code'].isin(opiod_codes).astype(int)

# pattern = '|'.join(opioids)
# flag = chem['NAME'].str.contains(pattern, case=False)
# #flag_map = flag.map({False: 0, True:1})
# opiod_codes = chem[flag]['CHEM SUB']
# scripts['opiods'] = 0
# scripts['opiods'] = [ 1 if scripts['bnf_code'][icode] in opiod_codes.to_list() else 0 
#                    for icode in range(len(scripts['bnf_code']))]

In [ ]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']
df_practices = pd.DataFrame(practices)

df_chem = pd.DataFrame(chem)
df_chem.drop_duplicates('CHEM SUB', inplace=True)
df_chem['NAME'] = df_chem['NAME'].str.lower()
df_chem['opioids'] = df_chem['NAME'].str.contains('|'.join(opioids) , case=False)

df_scripts = pd.DataFrame(scripts)
#df_scripts.drop_duplicates(inplace = True)

df_scripts_anomalies = df_scripts.merge(df_chem, left_on = 'bnf_code', right_on = 'CHEM SUB', how = 'left')
df_scripts_anomalies['opioids'] = df_scripts_anomalies['opioids'].fillna(0)
df_scripts_anomalies['opioids'] = df_scripts_anomalies['opioids'].astype(int)

df_scripts_anomalies.opioids.value_counts()

In [ ]:
df_scripts_anomalies.shape 
### Until here everything is right 

In [ ]:
#35 names in chem are in opioids.
#len(chem) becomes 3481 after dropping
#The total row after the merge is 973193
#duplicates (973193, 34843) are numbers for (total rows number, number of rows for opioids ) after merge (chem, script)

In [ ]:
df_scripts_anomalies.head()

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [ ]:
def calculate_opioids(df): 
    opioids_per_practice_mean = df.groupby('practice').opioids.mean()
    opioids_per_practice_count = df.groupby('practice').opioids.count()

    overall_mean = df['opioids'].mean()
    standard_dev = df['opioids'].std()
    #  I subtracted the μ obtained in (2), to get the relative prescription rate
    relative_opioids_per_practice = opioids_per_practice_mean -  overall_mean
    #calculating standard error
    standard_error_per_practice = standard_dev / (opioids_per_practice_count ** 0.5)
    #calculating z score
    opioid_scores = relative_opioids_per_practice/standard_error_per_practice
    opioid_scores_df = pd.DataFrame(opioid_scores).reset_index()
    return opioid_scores_df

In [ ]:
opioid_scores_df = calculate_opioids(df_scripts_anomalies)

In [ ]:
opioid_scores_df.head()

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [ ]:
df_scripts_anomalies_amount = df_scripts_anomalies.groupby('practice').agg({'opioids':'count'}).reset_index()
df_scripts_zscore_count = df_scripts_anomalies_amount.merge(opioid_scores_df, on = 'practice')
df_scripts_zscore_count.columns = ['practice', 'num_scripts', 'rare_zscore']

In [ ]:
df_scripts_zscore_count.sort_values(by = 'rare_zscore', ascending=False).head()

In [ ]:
df_scripts_zscore_count[df_scripts_zscore_count.practice == 'Y01852']

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
practices_grouped = practices.groupby('code').name.first().reset_index()

In [ ]:
final_merged = pd.merge(practices_grouped, df_scripts_zscore_count, left_on='code', right_on = 'practice')
final_mergedv2 = final_merged.sort_values(by = 'rare_zscore', ascending=False).reset_index(drop=True)
top100_v2 = list(zip(final_mergedv2.loc[0:99,'name'], 
                     final_mergedv2.loc[0:99,'rare_zscore'], 
                     final_mergedv2.loc[0:99, 'num_scripts']))
top100_v2[:10]

In [ ]:
#unique_practices = ...
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [ ]:
grader.score.dw__script_anomalies(top100_v2)

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [ ]:
scripts16 = pd.DataFrame(scripts2016)
scripts17 = pd.DataFrame(scripts)

In [ ]:
count_16 = scripts16.groupby('bnf_name').agg({'bnf_name':'count'})
count_16.columns = ['count_16']
count_16.head()

In [ ]:
count_17 = scripts17.groupby('bnf_name').agg({'bnf_code':'count'})
count_17.columns = ['count_17']
count_17.head()

In [ ]:
compare = pd.concat([count_16, count_17], axis=1, join='inner')
compare = compare[(compare['count_16']>=50)]
compare['growth_rate'] = (compare['count_17'] - compare['count_16']) / compare['count_16']
compare.sort_values('growth_rate', ascending=False, inplace=True)
compare.reset_index(inplace=True)

In [ ]:
compare.shape

In [ ]:
top50 = list(zip(compare.loc[0:49,'bnf_name'], compare.loc[0:49,'growth_rate'], 
                        compare.loc[0:49, 'count_16']))

bot50 = list(zip(compare.loc[compare.shape[0]-50:,'bnf_name'],
                 compare.loc[compare.shape[0]-50:,'growth_rate'],
                 compare.loc[compare.shape[0]-50:, 'count_16']))

script_growth = top50 + bot50

In [ ]:
## ANother way to do the same. 
# table = df_scripts17_num.merge(df_scripts16_num, on = 'bnf_name', how = 'inner')
# table['growth'] = (table['opioids17'] - table['opioids16']) / table['opioids16']
# table['growth'] = table['growth'].fillna(0)
# table.sort_values(by = 'growth', inplace = True, ascending = False)
# table_ = table.loc[table['opioids16'] >= 50]
# table_.reset_index(inplace=True)

In [ ]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [ ]:
grader.score.dw__script_growth(script_growth)

In [ ]:
scripts16 = pd.read_csv('dw-data/201606scripts_sample.csv.gz')
pct_growth = (scripts['bnf_name'].value_counts() / scripts16['bnf_name'].value_counts()) - 1
norm_pct_growth = pct_growth - (scripts['bnf_name'].count() / scripts16['bnf_name'].count() - 1)
norm_pct_growth.sort_values(ascending = False).head()
scripts16['bnf_name'].value_counts().head()
output = pd.concat([norm_pct_growth.rename('pct_growth'), scripts16['bnf_name'].value_counts().rename('count')], axis = 1)
output = output.dropna().sort_values('pct_growth', ascending = False).query('count >= 50')
most_extreme = pd.concat([output.head(50), output.tail(50)], axis = 0).reset_index()
script_growth = [(str(name), pct_growth, count) for name, pct_growth, count in most_extreme.iloc[0:100].values]

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [ ]:
scripts16 = pd.DataFrame(scripts2016)
scripts17 = pd.DataFrame(scripts)

In [ ]:
p = 1/len(scripts17.bnf_code.unique())
count_17_bnf_code = scripts17.groupby('bnf_code').agg({'bnf_code':'count'})
rates = count_17_bnf_code/len(scripts17['bnf_code'])
rates['rare'] = np.nan
rates.rare[rates.bnf_code < 0.1*p] = 1
rates = rates.fillna(0)
rates.columns = ['rates', 'rare']
rates = rates.reset_index()

In [ ]:
rates.head()

In [ ]:
list_rares = rates.bnf_code[rates.rare == 1.].to_list()
rare_codes = rates.bnf_code[rates.rare == 1.]
scripts17 = scripts17.merge(rates, on = 'bnf_code')
scripts17.head()

In [ ]:
rare_cost_df = scripts17[scripts17.rare == 1.].groupby(['practice', 'rare']).act_cost.sum()
rare_cost_df.head()

In [ ]:
rare_total_cost = rare_cost_df.sum()
total_act_cost = scripts17.act_cost.sum()
over_all_rare_cost_prop = rare_total_cost/total_act_cost

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
#1) First calculate cost sum per practice
act_cost_per_practice = scripts17.groupby('practice').agg({'act_cost':'sum'})
rare_cost_prop = scripts17[scripts17.rare == 1.].groupby('practice').agg({'act_cost':'sum'}) / act_cost_per_practice

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = rare_cost_prop.fillna(0) - over_all_rare_cost_prop

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
standard_errors = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
rare_scores = relative_rare_cost_prop/ standard_errors
rare_scores.columns = ['zscore_rare']
scripts_rare = pd.merge(scripts17, rare_scores.reset_index(), on = 'practice', how = 'left')

In [ ]:
scripts_rare.groupby('bnf_code').bnf_name.first().reset_index().head()

In [ ]:
scripts_rare_post_code = pd.merge(practices, scripts_rare, left_on='code', right_on = 'practice', how = 'left')

In [ ]:
scripts_rare_post_code[scripts_rare_post_code.code == 'Y03472'].groupby('code').first()

In [ ]:
scripts_rare_grouped = scripts_rare_post_code.groupby('code').first()

In [ ]:
scripts_rare_grouped = scripts_rare_grouped.sort_values(by = 'zscore_rare', ascending=False).reset_index()

In [ ]:
scripts_rare_grouped.head()

In [ ]:
top100 = list(zip(scripts_rare_grouped.loc[0:99,'code'], 
                  scripts_rare_grouped.loc[0:99,'name'], 
                  scripts_rare_grouped.loc[0:99, 'zscore_rare']
                 )
             )

In [ ]:
top100[:15]

In [ ]:
rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [ ]:
p = 1 / scripts['bnf_code'].nunique()

rates = scripts.groupby('bnf_code')['bnf_code'].count() / scripts['bnf_code'].count()
rates.name = 'rate'
rates = rates.fillna(0).reset_index()
rare_codes = list(rates[rates['rate'] < 0.1*p]['bnf_code'])


In [ ]:
rates.head()

In [ ]:
pattern = '|'.join(rare_codes)

scripts['rare'] = scripts['bnf_code'].str.contains(pattern, case=False)

rare_scripts = scripts[scripts['rare']==True]

rare_cost_prop = rare_scripts.groupby('practice')['act_cost'].sum() / scripts.groupby('practice')['act_cost'].sum()
relative_rare_cost_prop = rare_cost_prop.fillna(0) - rare_scripts['act_cost'].sum()/scripts['act_cost'].sum()


In [ ]:
standard_errors = relative_rare_cost_prop.std()

rare_scores = relative_rare_cost_prop/standard_errors
rare_scores = rare_scores.sort_values(ascending=False)

code_name = practices.groupby('code')['name'].first()

rare_scripts = [(key, code_name.loc[key], rare_scores[key]) for key in rare_scores.index]
rare_scripts = rare_scripts[:100]

In [ ]:
code_name

In [ ]:
grader.score.dw__rare_scripts(rare_scripts)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*